In [477]:
import pandas as pd

raw = pd.read_csv('./WtoDataCSV/WtoDataCSV.csv', encoding='latin1', low_memory=False)
raw.head()

,Indicator Category,Indicator Code,Indicator,Reporting Economy Code,Reporting Economy ISO3A Code,Reporting Economy,Partner Economy Code,Partner Economy ISO3A Code,Partner Economy,Product/Sector Classification Code,...,Period,Frequency Code,Frequency,Unit Code,Unit,Year,Value Flag Code,Value Flag,Text Value,Value
0,Merchandise trade - indices and prices,ITS_MTP_AM,Merchandise import value chained indices - annual,004,AFG,Afghanistan,000,NaN,World,SITC3,...,Annual,A,Annual,IDP,Previous year=100,2013,NaN,NaN,NaN,94.326
1,Merchandise trade - indices and prices,ITS_MTP_AM,Merchandise import value chained indices - annual,004,AFG,Afghanistan,000,NaN,World,SITC3,...,Annual,A,Annual,IDP,Previous year=100,2018,NaN,NaN,NaN,104.834
2,Merchandise trade - indices and prices,ITS_MTP_AM,Merchandise import value chained indices - annual,004,AFG,Afghanistan,000,NaN,World,SITC3,...,Annual,A,Annual,IDP,Previous year=100,2011,NaN,NaN,NaN,126.401
3,Merchandise trade - indices and prices,ITS_MTP_AM,Merchandise import value chained indices - annual,004,AFG,Afghanistan,000,NaN,World,SITC3,...,Annual,A,Annual,IDP,Previous year=100,2019,NaN,NaN,NaN,91.498
4,Merchandise trade - indices and prices,ITS_MTP_AM,Merchandise import value chained indices - annual,004,AFG,Afghanistan,000,NaN,World,SITC3,...,Annual,A,Annual,IDP,Previous year=100,2014,NaN,NaN,NaN,90.353


In [478]:
# 칼럼 걸러내기 : Indicator | Reporting Economy | Partner Economy | Product/Sector | Year | Value
temp_column = ['Indicator', 'Reporting Economy', 'Product/Sector', 'Partner Economy', 'Value', 'Year', 'Unit Code', 'Indicator Code']
df = raw.loc[:, temp_column]

# Value 칼럼 모두 Integer로 바꾸기 (소수 둘째 자리에서 반올림 하자)
df.Value = df.Value.round()
df.Value = df.Value.astype(int)
df

,Indicator,Reporting Economy,Product/Sector,Partner Economy,Value,Year,Unit Code,Indicator Code
0,Merchandise import value chained indices - annual,Afghanistan,Total merchandise,World,94,2013,IDP,ITS_MTP_AM
1,Merchandise import value chained indices - annual,Afghanistan,Total merchandise,World,105,2018,IDP,ITS_MTP_AM
2,Merchandise import value chained indices - annual,Afghanistan,Total merchandise,World,126,2011,IDP,ITS_MTP_AM
3,Merchandise import value chained indices - annual,Afghanistan,Total merchandise,World,91,2019,IDP,ITS_MTP_AM
4,Merchandise import value chained indices - annual,Afghanistan,Total merchandise,World,90,2014,IDP,ITS_MTP_AM
...,...,...,...,...,...,...,...,...
4960504,Commercial services exports by main sector  q...,Euro Area (19),Goods-related services,Extra Euro Area (19) Trade,13743,2019,USM,ITS_CS_QX
4960505,Commercial services exports by main sector  q...,Euro Area (19),Goods-related services,Extra Euro Area (19) Trade,8776,2013,USM,ITS_CS_QX
4960506,Commercial services exports by main sector  q...,Euro Area (19),Goods-related services,Extra Euro Area (19) Trade,8231,2010,USM,ITS_CS_QX
4960507,Commercial services exports by main sector  q...,Euro Area (19),Goods-related services,Extra Euro Area (19) Trade,8007,2012,USM,ITS_CS_QX


In [479]:
# Partner Economy == World 골라내기 및 Product/Sector 골라내기
condition_1 = df['Product/Sector'] == 'Total merchandise'
condition_2 = df['Product/Sector'] == 'Fuels and mining products'
df = df.loc[condition_1 | condition_2]

In [480]:
# df['Unit Code'] == 'USM' 골라내기 및 df['Indicator Code'] == 'ITS_MTV_AX' 골라내기
# 이 과정을 거쳐야만 국가별 데이터가 2개씩 남게됨 (전체데이터, 원자재 데이터)
condition_5 = df['Unit Code'] == 'USM'
condition_6 = df['Indicator Code'] == 'ITS_MTV_AX'
df = df.loc[condition_5 & condition_6]

In [481]:
# Year == 1990 ~ 2019 외 행 제거
condition_3 = df['Year'] >= 1990
condition_4 = df['Year'] < 2020
df = df.loc[condition_3 & condition_4]

In [482]:
# 데이터가 잘 나오는지 테스트
temp_df = df.loc[(df['Year']==2019) & (df['Reporting Economy']=='Argentina')]
temp_df

,Indicator,Reporting Economy,Product/Sector,Partner Economy,Value,Year,Unit Code,Indicator Code
139492,Merchandise exports by product group  annual,Argentina,Fuels and mining products,World,2629,2019,USM,ITS_MTV_AX
139535,Merchandise exports by product group  annual,Argentina,Total merchandise,World,65116,2019,USM,ITS_MTV_AX


In [483]:
# 피봇 데이터를 통해 연도를 칼럼에 나열
pivot_df = df.pivot(index=['Reporting Economy', 'Product/Sector', 'Partner Economy'], \
    columns='Year', values='Value')
pivot_df.reset_index(inplace = True)
df = pivot_df
# df.rename(columns={'Year' : 'Index'}, inplace = True)

In [484]:
df.reset_index(inplace=True)

In [485]:
df.set_index('index', inplace=True)

In [486]:
df

Year,Reporting Economy,Product/Sector,Partner Economy,1990,1991,1992,1993,1994,1995,1996,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
index,,,,,,,,,,,,,,,,,,,,,
0,Afghanistan,Fuels and mining products,World,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,NaN,NaN,NaN,26.0,7.0,51.0,100.0,89.0
1,Afghanistan,Total merchandise,World,235.0,189.0,181.0,688.0,105.0,166.0,128.0,...,388.0,376.0,429.0,515.0,570.0,571.0,596.0,723.0,875.0,864.0
2,Africa,Total merchandise,World,105090.0,99501.0,97320.0,92868.0,96728.0,111984.0,125415.0,...,519184.0,612722.0,637066.0,594337.0,564907.0,397667.0,363879.0,434666.0,509726.0,478649.0
3,"Africa, CIS and Middle East",Total merchandise,World,NaN,NaN,293794.0,285690.0,322572.0,376617.0,432615.0,...,1963277.0,2596998.0,2731039.0,2661626.0,2509698.0,1748852.0,1563492.0,1848092.0,2273324.0,2113769.0
4,"African, Caribbean and Pacific States (ACP)",Total merchandise,World,82481.0,76139.0,76019.0,74954.0,78037.0,91513.0,101920.0,...,389086.0,497027.0,481954.0,465517.0,464719.0,334304.0,299169.0,356007.0,410473.0,386135.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
473,"Yugoslavia, Socialist Federal Republic of",Total merchandise,World,14308.0,13947.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
474,Zambia,Fuels and mining products,World,NaN,NaN,NaN,739.0,NaN,933.0,NaN,...,5992.0,7041.0,6688.0,7317.0,7467.0,5015.0,4649.0,6371.0,7021.0,5253.0
475,Zambia,Total merchandise,World,1309.0,1083.0,756.0,826.0,927.0,1040.0,1037.0,...,7200.0,9001.0,9365.0,10607.0,9694.0,6607.0,6372.0,8000.0,9034.0,7047.0


In [487]:
# Product/Sector : Total merchandise, Fuels and mining products
condition_7 = df['Product/Sector'] == 'Total merchandise'
condition_8 = df['Product/Sector'] == 'Fuels and mining products'
total_df = df.loc[condition_7]
fuels_and_mining_df = df.loc[condition_8]


In [488]:
total_df.reset_index(inplace=True)
total_df.set_index('index', inplace=True)

In [489]:
total_df

Year,Reporting Economy,Product/Sector,Partner Economy,1990,1991,1992,1993,1994,1995,1996,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
index,,,,,,,,,,,,,,,,,,,,,
1,Afghanistan,Total merchandise,World,235.0,189.0,181.0,688.0,105.0,166.0,128.0,...,388.0,376.0,429.0,515.0,570.0,571.0,596.0,723.0,875.0,864.0
2,Africa,Total merchandise,World,105090.0,99501.0,97320.0,92868.0,96728.0,111984.0,125415.0,...,519184.0,612722.0,637066.0,594337.0,564907.0,397667.0,363879.0,434666.0,509726.0,478649.0
3,"Africa, CIS and Middle East",Total merchandise,World,NaN,NaN,293794.0,285690.0,322572.0,376617.0,432615.0,...,1963277.0,2596998.0,2731039.0,2661626.0,2509698.0,1748852.0,1563492.0,1848092.0,2273324.0,2113769.0
4,"African, Caribbean and Pacific States (ACP)",Total merchandise,World,82481.0,76139.0,76019.0,74954.0,78037.0,91513.0,101920.0,...,389086.0,497027.0,481954.0,465517.0,464719.0,334304.0,299169.0,356007.0,410473.0,386135.0
6,Albania,Total merchandise,World,230.0,100.0,70.0,122.0,140.0,202.0,207.0,...,1544.0,1947.0,1967.0,2339.0,2429.0,1917.0,1959.0,2292.0,2870.0,2716.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469,West African Economic and Monetary Union (WAEMU),Total merchandise,World,5201.0,4715.0,4903.0,4608.0,4959.0,6626.0,7422.0,...,20693.0,24031.0,23785.0,24650.0,26424.0,23267.0,22969.0,25390.0,28339.0,29202.0
471,Yemen,Total merchandise,World,692.0,659.0,619.0,611.0,934.0,1945.0,2675.0,...,8100.0,9700.0,8300.0,8300.0,7792.0,1911.0,651.0,1244.0,1358.0,1435.0
473,"Yugoslavia, Socialist Federal Republic of",Total merchandise,World,14308.0,13947.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [490]:
fuels_and_mining_df.reset_index(inplace=True)
fuels_and_mining_df.set_index('index', inplace=True)

In [491]:
fuels_and_mining_df

Year,Reporting Economy,Product/Sector,Partner Economy,1990,1991,1992,1993,1994,1995,1996,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
index,,,,,,,,,,,,,,,,,,,,,
0,Afghanistan,Fuels and mining products,World,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,NaN,NaN,NaN,26.0,7.0,51.0,100.0,89.0
5,Albania,Fuels and mining products,World,NaN,NaN,NaN,NaN,NaN,NaN,31.0,...,477.0,645.0,738.0,951.0,817.0,505.0,398.0,390.0,551.0,473.0
7,Algeria,Fuels and mining products,World,12530.0,12239.0,10795.0,9838.0,8649.0,9803.0,12509.0,...,56253.0,72400.0,70893.0,63988.0,58515.0,33326.0,28661.0,33881.0,39603.0,33678.0
12,Angola,Fuels and mining products,World,3899.0,3399.0,3451.0,3733.0,2902.0,3522.0,4780.0,...,49283.0,65762.0,69428.0,67661.0,58617.0,32250.0,26574.0,33909.0,38769.0,34298.0
15,Antigua and Barbuda,Fuels and mining products,World,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,5.0,2.0,1.0,1.0,0.0,1.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
463,Viet Nam,Fuels and mining products,World,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8714.0,11816.0,12082.0,10623.0,10419.0,6287.0,5034.0,7030.0,6686.0,6530.0
470,Yemen,Fuels and mining products,World,NaN,183.0,NaN,NaN,NaN,1505.0,NaN,...,5683.0,6141.0,6339.0,6000.0,5762.0,1404.0,326.0,797.0,1014.0,1105.0
472,"Yugoslavia, Socialist Federal Republic of",Fuels and mining products,World,1623.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
